#DATASCI W261: Machine Learning at Scale 

* **Sayantan Satpati**
* **sayantan.satpati@ischool.berkeley.edu**
* **W261**
* **Week-3**
* **Assignment-4**
* **Date of Submission: 22-SEP-2015**

#  === Week 3 Hadoop & Apriori ===

**HW3.0. What is a merge sort? Where is it used in Hadoop?**



In [54]:
%%writefile mapper_hw31.py

#!/usr/bin/env python

import sys
import re
import itertools

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        items = re.split(r'\s+', line)
        #Sort the list
        items.sort()
        
        for c in itertools.combinations(items, 1):
            print '%s,%s\t%d\t%d' %(c[0], '*', 1, len(items))
            
        for c in itertools.combinations(items, 2):
            print '%s,%s\t%d' %(c[0], c[1], 1)
        
        ''' Commenting out itemset-3 for the moment
        for c in itertools.combinations(items, 3):
            print '%s,%s,%s\t%d' %(c[0], c[1], c[2], 1)
        '''
    except Exception as e:
        print e

Overwriting mapper_hw31.py


In [55]:
!chmod a+x mapper_hw31.py

In [60]:
%%writefile reducer_hw31.py
#!/usr/bin/python
import sys
import re
import heapq
from sets import Set

'''
a1,* 1 4
a1,* 1 5
a1,b1 1
a1,b1 1
a1,b2 1
a1,b2 1
a2,* 1 6
'''

itemset_1_cnt = 0
itemset_2_cnt = 0

itemset_1_last = None
itemset_2_last = None

THRESHOLD = 100

# Statistics
# Unique Items
uniq = Set()
# Max Basket Length
max_basket_len = 0
# Total Itemset Counts for Sizes: 1 & 2
total_itemset_1 = 0
total_itemset_2 = 0

for line in sys.stdin:
    # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        tokens = re.split(r'\s+', line)
    
        # Split the key by <COMMA> delimeter
        items = tokens[0].split(",")
        i1 = items[0]
        i2 = items[1]
        
        # Count
        count = int(tokens[1])
        
        if itemset_1_last != i1:
            uniq.add(i1)
            
            basket_len = int(tokens[2])
            if basket_len >= max_basket_len:
                max_basket_len = basket_len
                
            if itemset_1_cnt >= THRESHOLD:
                total_itemset_1 += 1
                
            # Reset
            itemset_1_last = i1
            itemset_1_cnt = count
            itemset_2_last = None
            itemset_2_cnt = 0
        else:
            if i2 == '*':
                itemset_1_cnt += count
                
                basket_len = int(tokens[2])
                if basket_len > max_basket_len:
                    max_basket_len = basket_len
            else:
                if itemset_2_last != tokens[0]:
                    if itemset_1_cnt >= THRESHOLD and itemset_2_cnt >= THRESHOLD:
                        total_itemset_2 += 1
                        
                    itemset_2_last = tokens[0]
                    itemset_2_cnt = count
                else:
                    itemset_2_cnt += count
                    
# Last Set of Counts
if itemset_1_cnt >= THRESHOLD:
    total_itemset_1 += 1
if itemset_1_cnt >= THRESHOLD and itemset_2_cnt >= THRESHOLD:
    total_itemset_2 += 1

print '=== Statistics ==='
print 'Total Unique Items: %d' %(len(uniq))
print 'Maximum Basket Length: %d' %(max_basket_len)
print 'Total # itemsets of size 1: %d' %(total_itemset_1)
print 'Total # itemsets of size 2: %d' %(total_itemset_2)
        
        

Overwriting reducer_hw31.py


In [86]:
%%writefile reducer_hw31.py
#!/usr/bin/python
import sys
import re
import heapq
from sets import Set

'''
a1,* 1 4
a1,* 1 5
a1,b1 1
a1,b1 1
a1,b2 1
a1,b2 1
a2,* 1 6
'''

itemset_1_cnt = 0
itemset_2_cnt = 0

itemset_1_last = None

THRESHOLD = 100

# Statistics
# Unique Items
uniq = Set()
# Max Basket Length
max_basket_len = 0
# Total Itemset Counts for Sizes: 1 & 2
total_itemset_1 = 0
total_itemset_2 = 0

d_counts = {}

def update_counts():
    global itemset_1_last
    global d_counts
    global total_itemset_1
    global total_itemset_2
    key = '{0},{1}'.format(itemset_1_last, '*')
    if d_counts.get(key, 0) >= THRESHOLD:
        total_itemset_1 += 1
        for k,v in d_counts.iteritems():
            if k != key:
                if v >= THRESHOLD:
                    total_itemset_2 += 1

for line in sys.stdin:
    # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        tokens = re.split(r'\s+', line)
    
        # Split the key by <COMMA> delimeter
        items = tokens[0].split(",")
        i1 = items[0]
        i2 = items[1]
        
        # Count
        count = int(tokens[1])
        
        if not itemset_1_last:
            itemset_1_last = i1
        
        if itemset_1_last != i1:
            # Emit Contents of Dict
            update_counts()
            
            if i2 == '*':
                uniq.add(i1)
                basket_len = int(tokens[2])
                if basket_len > max_basket_len:
                    max_basket_len = basket_len
                            
            d_counts.clear()
            itemset_1_last = i1
        else:
            key = tokens[0]
            d_counts[key] = d_counts.get(key, 0) + count
            
            if i2 == '*':
                uniq.add(i1)
                basket_len = int(tokens[2])
                if basket_len > max_basket_len:
                    max_basket_len = basket_len
                    
# Last Record
update_counts()
                    
print '=== Statistics ==='
print 'Total Unique Items: %d' %(len(uniq))
print 'Maximum Basket Length: %d' %(max_basket_len)
print 'Total # itemsets of size 1: %d' %(total_itemset_1)
print 'Total # itemsets of size 2: %d' %(total_itemset_2)
        
        

Overwriting reducer_hw31.py


In [88]:
!chmod a+x reducer_hw31.py

In [89]:
'''
HW3.1. Product Recommendations
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk3/hw31/output

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D map.output.key.field.separator=, \
-D mapred.text.key.partitioner.options=-k1,1 \
-input /user/cloudera/w261/wk3/hw31/input/ProductPurchaseData.txt \
-output /user/cloudera/w261/wk3/hw31/output \
-file ./mapper_hw31.py \
-mapper 'python mapper_hw31.py' \
-file ./reducer_hw31.py \
-reducer 'python reducer_hw31.py'

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk3/hw31/output/part-00000

Deleted /user/cloudera/w261/wk3/hw31/output
packageJobJar: [./mapper_hw31.py, ./reducer_hw31.py, /tmp/hadoop-cloudera/hadoop-unjar4802976138694835527/] [] /tmp/streamjob371666725523987824.jar tmpDir=null
15/09/20 12:15:58 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/20 12:15:58 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/20 12:15:59 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/20 12:15:59 INFO streaming.StreamJob: Running job: job_201509191045_0012
15/09/20 12:15:59 INFO streaming.StreamJob: To kill this job, run:
15/09/20 12:15:59 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509191045_0012
15/09/20 12:15:59 INFO streaming.StreamJob: Tracking URL: http://0.0.0.0:50030/jobdetails.jsp?jobid=job_201509191045_0012
15/09/20 12:16:00 INFO streaming.StreamJob:  map 0%  reduce 0%
15/09

In [93]:
%%writefile mapper_hw32.py

#!/usr/bin/env python

import sys
import re
import itertools

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        items = re.split(r'\s+', line)
        #Sort the list
        items.sort()
        
        l = len(items)
        
        for i in xrange(l):
            print '%s,*\t%d' %(items[i], 1)
            for j in xrange(i+1, l):
               print '%s,%s\t%d' %(items[i], items[j], 1) 
    except Exception as e:
        print e

Overwriting mapper_hw32.py


In [94]:
!chmod a+x mapper_hw32.py

In [98]:
%%writefile reducer_hw32.py
#!/usr/bin/python
import sys
import re
import heapq

itemset_1_cnt = 0
itemset_2_cnt = 0

itemset_1_last = None
itemset_2_last = None

'''
a1,* 1
a1,* 1
a1,b1 1
a1,b1 1
a1,b2 1
a1,b2 1
a2,* 1
'''

THRESHOLD = 100
# Store Itemsets 2
dict = {}

for line in sys.stdin:
    # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        tokens = re.split(r'\s+', line)
    
        # Split the key by <COMMA> delimeter
        items = tokens[0].split(",")
        i1 = items[0]
        i2 = items[1]
        
        if not itemset_1_last:
            itemset_1_last = i1
        
        if itemset_1_last != i1:
            '''
            if itemset_1_cnt >= THRESHOLD:
                confidence = (itemset_2_cnt * 1.0) / itemset_1_cnt
                print '[%d,%d]%s\t%f' %(itemset_1_cnt, itemset_2_cnt, tokens[0], confidence)
                dict[tokens[0]] = confidence
            '''
                        
            # Reset
            itemset_1_last = i1
            itemset_1_cnt = int(tokens[1])
            itemset_2_last = None
            itemset_2_cnt = 0
        else:
            if i2 == '*':
                itemset_1_cnt += int(tokens[1])
            else:
                if itemset_2_last != tokens[0]:
                    if itemset_1_cnt >= THRESHOLD and itemset_2_cnt >= THRESHOLD:
                        confidence = (itemset_2_cnt * 1.0) / itemset_1_cnt
                        #print '[%d,%d]%s\t%f' %(itemset_1_cnt, itemset_2_cnt, itemset_2_last, confidence)
                        dict[itemset_2_last] = confidence
                    itemset_2_last = tokens[0]
                    itemset_2_cnt = int(tokens[1]) 
                else:
                    itemset_2_cnt += int(tokens[1])                    

# Last Set of Counts
if itemset_1_cnt >= THRESHOLD and itemset_2_cnt >= THRESHOLD:
    confidence = (itemset_2_cnt * 1.0) / itemset_1_cnt
    #print '[%d,%d]%s\t%f' %(itemset_1_cnt, itemset_2_cnt, itemset_2_last, confidence)
    dict[itemset_2_last] = confidence

print '=== Top 5 Confidence ==='
sorted_dict = sorted(dict.items(), key=lambda x:(-x[1], x[0]))
for j,k in sorted_dict[:5]:
    print '%s\t%f' %(j,k)
        
        
        

Overwriting reducer_hw32.py


In [99]:
!chmod a+x reducer_hw32.py

In [100]:
'''
HW3.2. Confidence Caculations
List the top 5 rules with corresponding confidence scores in decreasing order of confidence score 
for frequent (100>count) itemsets of size 2
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk3/hw32/output

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D map.output.key.field.separator=, \
-D mapred.text.key.partitioner.options=-k1,1 \
-input /user/cloudera/w261/wk3/hw32/input/ProductPurchaseData.txt \
-output /user/cloudera/w261/wk3/hw32/output \
-file ./mapper_hw32.py \
-mapper 'python mapper_hw32.py' \
-file ./reducer_hw32.py \
-reducer 'python reducer_hw32.py'

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk3/hw32/output/part-00000

Deleted /user/cloudera/w261/wk3/hw32/output
packageJobJar: [./mapper_hw32.py, ./reducer_hw32.py, /tmp/hadoop-cloudera/hadoop-unjar8486018449480148677/] [] /tmp/streamjob286663036493526101.jar tmpDir=null
15/09/20 13:08:58 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/20 13:08:58 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/20 13:08:58 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/20 13:08:58 INFO streaming.StreamJob: Running job: job_201509191045_0014
15/09/20 13:08:58 INFO streaming.StreamJob: To kill this job, run:
15/09/20 13:08:58 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509191045_0014
15/09/20 13:08:59 INFO streaming.StreamJob: Tracking URL: http://0.0.0.0:50030/jobdetails.jsp?jobid=job_201509191045_0014
15/09/20 13:09:00 INFO streaming.StreamJob:  map 0%  reduce 0%
15/09

In [28]:
'''
HW2.1. Sort in Hadoop MapReduce
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw21/output

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D  mapred.text.key.comparator.options=-n \
-input /user/cloudera/w261/wk2/hw21/input/random.txt \
-output /user/cloudera/w261/wk2/hw21/output \
-file ./identity_map_red_hw21.py \
-mapper ./identity_map_red_hw21.py \
-file ./identity_map_red_hw21.py \
-reducer ./identity_map_red_hw21.py

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw21/output/part-00000 | head -10

Deleted /user/cloudera/w261/wk2/hw21/output
File: /home/cloudera/dev/ml/w261/wk3/./identity_map_red_hw21.py does not exist, or is not readable.
Streaming Command Failed!
cat: `/user/cloudera/w261/wk2/hw21/output/part-00000': No such file or directory


In [6]:
%%writefile mapper_hw22.py

#!/usr/bin/env python

import sys
import re

def strip_special_chars(word):
    return re.sub('[^A-Za-z0-9]+', '', word)

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        email = re.split(r'\t+', line)

        # Check whether Content is present
        if len(email) < 4:
            continue

        # Get the content as a list of words
        content = email[len(email) - 1].split()

        for w in content:
            w = strip_special_chars(w)
            if w == 'assistance':
                print '%s\t%d' % (w, 1)
    except Exception as e:
        print line
        print e

Overwriting mapper_hw22.py


In [7]:
!chmod a+x mapper_hw22.py

In [8]:
%%writefile reducer_hw22.py

#!/usr/bin/env python

import sys
import re

word = None
count = 0

for line in sys.stdin:
    # Remove leading & trailing chars
    line = line.strip()
    # Split the line by <TAB> delimeter
    wc = re.split(r'\t+', line)
    
    word = wc[0]
    count += int(wc[1])
    
print '%s\t%d' % (word, count)

Overwriting reducer_hw22.py


In [9]:
!chmod a+x reducer_hw22.py

In [10]:
'''
HW2.2. Using the Enron data from HW1 and Hadoop MapReduce streaming, 
write mapper/reducer pair that  will determine the number of occurrences of a single, 
user-specified word. Examine the word “assistance” and report your results.
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw22/output

!ls -l *hw22.py

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.reduce.tasks = 1 \
-input /user/cloudera/w261/wk2/hw22/input/enronemail_1h.txt \
-output /user/cloudera/w261/wk2/hw22/output \
-file ./mapper_hw22.py \
-mapper 'python mapper_hw22.py' \
-file ./reducer_hw22.py \
-reducer 'python reducer_hw22.py'

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw22/output/part-00000

Deleted /user/cloudera/w261/wk2/hw22/output
-rwxrwxr-x 1 cloudera cloudera 695 Sep 14 12:14 mapper_hw22.py
-rwxrwxr-x 1 cloudera cloudera 307 Sep 14 12:14 reducer_hw22.py
packageJobJar: [./mapper_hw22.py, ./reducer_hw22.py, /tmp/hadoop-cloudera/hadoop-unjar4404270376137637736/] [] /tmp/streamjob6249198798003576990.jar tmpDir=null
15/09/14 12:14:26 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/14 12:14:26 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/14 12:14:27 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/14 12:14:27 INFO streaming.StreamJob: Running job: job_201509131822_0042
15/09/14 12:14:27 INFO streaming.StreamJob: To kill this job, run:
15/09/14 12:14:27 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509131822_0042
15/09/14 12:14:27 INFO streaming.StreamJob: Tracking URL: http

#### HW2.2: The word assistance has 9 occurrences if we perform the following steps:
1. Tokenization
2. Remove Special Chars
3. Don't include emails which have a bad format (3 cols as opposed to 4 cols)
4. Include only Email Content

In [25]:
%%writefile mapper_hw23.py

#!/usr/bin/env python

import sys
import os
import re

# Output from mapp
vocab = set()
word_counts = {
    "1": {},
    "0": {}
}
total = 0
total_spam = 0
total_ham = 0

word_list = os.environ['WORDS'].split(",")

def strip_special_chars(word):
    word = word.strip().lower()
    return re.sub('[^A-Za-z0-9]+', '', word)

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        email = re.split(r'\t+', line)

        # Check whether Content is present
        if len(email) < 4:
            continue

        # Get the content as a list of words
        spam = email[1]
        content = email[len(email) - 1].split()
        
        # Totals
        total += 1
        if spam == '1':
            total_spam += 1
        else:
            total_ham += 1

        for w in content:
            w = strip_special_chars(w)
            
            # Add to category dict
            word_counts[spam][w] = word_counts[spam].get(w, 0) + 1
                
            # Vocab Unique
            vocab.add(w)
    except Exception as e:
        print line
        print e
        
print 'TOTAL_DOCUMENTS\t%d\t%d\t%d' % (total,total_spam,total_ham)
print 'TOTAL_WORDS\t%d\t%d\t%d' % (len(vocab), sum(word_counts['1'].values()), sum(word_counts['0'].values()))
for w in word_list:
    print '%s\t%d\t%d' %(w, word_counts['1'].get(w, 0.0), word_counts['0'].get(w, 0.0))

Overwriting mapper_hw23.py


In [26]:
!chmod a+x mapper_hw23.py

In [29]:
%%writefile reducer_hw23.py

#!/usr/bin/env python

import sys
import os
import re
import math

# Totals from Mapper
total = 0
total_spam = 0
total_ham = 0

vocab = 0
vocab_spam = 0
vocab_ham = 0
word_count = {}

word_list = os.environ['WORDS'].split(",")

for line in sys.stdin:
    try:
         # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        tokens = re.split(r'\t+', line)
        
        if tokens[0] == 'TOTAL_DOCUMENTS':
            total += int(tokens[1])
            total_spam += int(tokens[2])
            total_ham += int(tokens[3])
        elif tokens[0] == 'TOTAL_WORDS':
            vocab = int(tokens[1])
            vocab_spam = int(tokens[2])
            vocab_ham = int(tokens[3])
        else:
            word_count[tokens[0]] = (int(tokens[1]), int(tokens[2]))
    except Exception as e:
        sys.exit(1)

prior_spam = (total_spam * 1.0) / total
prior_ham = (total_ham * 1.0) / total

spam_lhood_denom = vocab_spam + vocab
ham_lhood_denom = vocab_ham + vocab
spam_lhood_log = 0.0
ham_lhood_log = 0.0
for w in word_list:
    spam_lhood_log += math.log( (word_count[w][0] + 1.0) * 1.0 / spam_lhood_denom )
    ham_lhood_log += math.log( (word_count[w][1] + 1.0) * 1.0 / ham_lhood_denom )
spam_score = spam_lhood_log + math.log(prior_spam)
ham_score = ham_lhood_log + math.log(prior_ham)

classification = 'HAM'
if spam_score > ham_score:
    classification = 'SPAM'
   
print '#<Feature>\t<Spam_Score>\t<Ham_Score>\t<Predicted_Class>'
print '%s\t%f\t%f\t%s' %(",".join(word_list), spam_score, ham_score, classification)

Overwriting reducer_hw23.py


In [30]:
!chmod a+x reducer_hw23.py

In [32]:
'''
HW2.3. Using the Enron data from HW1 and Hadoop MapReduce, write  a mapper/reducer pair that
   will classify the email messages by a single, user-specified word. 
   Examine the word “assistance” and report your results.
   
RESULT: The document is classified as a SPAM.
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw23/output

!ls -l *hw23.py

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.reduce.tasks = 1 \
-input /user/cloudera/w261/wk2/hw23/input/enronemail_1h.txt \
-output /user/cloudera/w261/wk2/hw23/output \
-file ./mapper_hw23.py \
-mapper 'python mapper_hw23.py' \
-file ./reducer_hw23.py \
-reducer 'python reducer_hw23.py' \
-cmdenv WORDS='assistance' \

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw23/output/part-00000

Deleted /user/cloudera/w261/wk2/hw23/output
-rwxrwxr-x 1 cloudera cloudera 1464 Sep 14 13:24 mapper_hw23.py
-rwxrwxr-x 1 cloudera cloudera 1582 Sep 14 13:26 reducer_hw23.py
packageJobJar: [./mapper_hw23.py, ./reducer_hw23.py, /tmp/hadoop-cloudera/hadoop-unjar2678392748320022312/] [] /tmp/streamjob7567663044371259044.jar tmpDir=null
15/09/14 13:27:09 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/14 13:27:09 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/14 13:27:10 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/14 13:27:10 INFO streaming.StreamJob: Running job: job_201509131822_0054
15/09/14 13:27:10 INFO streaming.StreamJob: To kill this job, run:
15/09/14 13:27:10 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509131822_0054
15/09/14 13:27:10 INFO streaming.StreamJob: Tracking URL: ht

#### HW2.3: Document is classified as a SPAM 

In [ ]:
'''
HW2.4. Using the Enron data from HW1 and in the Hadoop MapReduce framework, write  a mapper/reducer pair that
   will classify the email messages using multinomial Naive Bayes Classifier using a list of one or more 
   user-specified words.
   (SAME MAPPER AND REDUCER AS IN HW2.3 IS USED, BUT WITH DIFFERENT PARAMETERS PASSED IN -cmdenv)
   
RESULT: The document is classified as a SPAM.
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk2/hw24/output

!ls -l *hw23.py

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D mapred.reduce.tasks = 1 \
-input /user/cloudera/w261/wk2/hw24/input/enronemail_1h.txt \
-output /user/cloudera/w261/wk2/hw24/output \
-file ./mapper_hw23.py \
-mapper 'python mapper_hw23.py' \
-file ./reducer_hw23.py \
-reducer 'python reducer_hw23.py' \
-cmdenv WORDS='assistance,valium,enlargementWithATypo' \

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk2/hw24/output/part-00000

#### HW2.4: Document is classified as a SPAM 